In [1]:
!pip install langchain crewai transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.3 MB/s eta 0:00

In [2]:
import json
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms.base import LLM
from crewai import Agent, Task, Crew, Process

In [11]:
# Sample: multiple leads
leads = [
    {
        "name": "Jane Doe",
        "job_title": "Head of Sales",
        "company": "Acme Corp",
        "interests": ["B2B SaaS", "lead generation", "CRM automation"]
    },
    {
        "name": "John Smith",
        "job_title": "Marketing Director",
        "company": "Beta Inc",
        "interests": ["growth hacking", "email marketing", "ABM"]
    }
]

results = []
for lead in leads:
    prompt = build_email_prompt(lead, product)
    email = llm_executor(prompt)
    match = re.search(r"Subject:\s*(.*?)\nBody:\s*([\s\S]*)", email)
    subject = match.group(1).strip() if match else ""
    body = match.group(2).strip() if match else email.strip()
    results.append({"Name": lead['name'], "Subject": subject, "Body": body})

import pandas as pd
df = pd.DataFrame(results)
df.to_csv("generated_emails.csv", index=False)
df

,Name,Subject,Body
0,Jane Doe,,```\nSubject: Re: Acme Corp Sales Lead Inquiry...
1,John Smith,,---\nSubject: Unlocking hidden growth potenti...


In [3]:
# Simulated leads and product info
sample_data = {
    "lead": {
        "name": "Jane Doe",
        "job_title": "Head of Sales",
        "company": "Acme Corp",
        "interests": ["B2B SaaS", "lead generation", "CRM automation"]
    },
    "product": {
        "name": "SalesAI Automation Suite",
        "features": [
            "AI-driven lead enrichment",
            "Seamless CRM integration",
            "Automated personalized outreach"
        ],
        "benefits": [
            "Save 10+ hours/week per SDR",
            "Boost response rates by 30%",
            "Instant LinkedIn-to-inbox flows"
        ]
    }
}

# Optionally, write to/read from JSON file for modularity
with open("lead_product_sample.json", "w") as f:
    json.dump(sample_data, f, indent=2)

# For demonstration, we'll read it just like a data handler module
with open("lead_product_sample.json", "r") as f:
    data = json.load(f)
lead = data['lead']
product = data['product']

In [6]:
# Optional: Authenticate, if needed (for private models)
# login('your_huggingface_token')

model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Build a generation pipeline
gen_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

class Gemma1BItLLM(LLM):
    def _call(self, prompt, stop=None):
        output = gen_pipe(prompt)[0]['generated_text']
        # The pipeline includes input "prompt" at start, extract only generated part
        return output[len(prompt):].strip()

    @property
    def _llm_type(self) -> str:
        return "custom-gemma-1b-it"

llm = Gemma1BItLLM()

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `devs` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushi

In [7]:
def build_email_prompt(lead, product):
    prompt = f"""You are an expert SDR assistant.

Given the following lead and product information, generate a highly personalized cold email:

Lead:
- Name: {lead['name']}
- Job Title: {lead['job_title']}
- Company: {lead['company']}
- Interests: {', '.join(lead['interests'])}

Product:
- Name: {product['name']}
- Features: {', '.join(product['features'])}
- Benefits: {', '.join(product['benefits'])}

Requirements:
1. Create an engaging subject line directly addressing the lead's pain points or goals.
2. Write a brief, conversational body making the case for why your product could help {lead['company']}.
3. Reference both lead's interests and product benefits.
4. Close with a call-to-action for a short intro call.

Format:
Subject: <Your Subject Line>
Body:
<Your Email Body>

### Output:
"""
    return prompt

In [8]:
# Define the agent using CrewAI
reasoning_agent = Agent(
    role="Email Personalization Reasoner",
    goal="Analyze lead and product data to craft optimized sales email",
    backstory="Handles rapid, high-quality outreach for a SaaS sales team.",
    verbose=True,
    allow_delegation=False
)

# The task will use the prompt builder
email_task = Task(
    description="Generate personalized sales email with engaging subject and body.",
    agent=reasoning_agent,
    expected_output="A two-part result: (1) subject line, (2) personalized email body.",
    output_format="Markdown"
)

crew = Crew(
    agents=[reasoning_agent],
    tasks=[email_task],
    process=Process.sequential, # Email writing is a single step now
)

In [9]:
# Compose prompt with our data
email_prompt = build_email_prompt(lead, product)

# CrewAI: Assign LLM as the execution tool for the agent (LangChain interface)
def llm_executor(prompt):
    return llm(prompt)

# For demonstration, we call our executor directly.
email_output = llm_executor(email_prompt)

print(email_output)

<ipython-input-9-a14eb73f3c72>:6: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(prompt)


---
Subject: Drowning in Leads? SalesAI Automation Suite Can Help Acme Corp.

Body:
Acme Corp, I've been following your work at Acme Corp with interest, and I understand the challenges of managing a growing sales team and maintaining lead generation. Many sales teams struggle with time constraints and manually chasing leads, often leading to missed opportunities.

SalesAI Automation Suite is designed to address these challenges. It's an AI-driven lead enrichment tool that automatically connects with potential leads on LinkedIn and other sources, providing valuable insights and rich data.  This means you can save 10+ hours per week per SDR, allowing them to focus on more strategic sales activities and boosting response rates by 30%.  

Our seamless CRM integration works with Salesforce, HubSpot, and other systems, streamlining your lead flow.  Furthermore, our automated personalized outreach helps you instantly connect with prospects on LinkedIn, ensuring a more engaging and effective a

In [10]:
import re

# Extract nicely if the model outputs with markers as expected
match = re.search(
    r"Subject:\s*(.*?)\nBody:\s*([\s\S]*)", email_output
)
if match:
    subject = match.group(1).strip()
    body = match.group(2).strip()
else:
    subject = "N/A"
    body = email_output.strip()

print(f"---\nSubject: {subject}\n\nBody:\n{body}\n---")

---
Subject: N/A

Body:
---
Subject: Drowning in Leads? SalesAI Automation Suite Can Help Acme Corp.

Body:
Acme Corp, I've been following your work at Acme Corp with interest, and I understand the challenges of managing a growing sales team and maintaining lead generation. Many sales teams struggle with time constraints and manually chasing leads, often leading to missed opportunities.

SalesAI Automation Suite is designed to address these challenges. It's an AI-driven lead enrichment tool that automatically connects with potential leads on LinkedIn and other sources, providing valuable insights and rich data.  This means you can save 10+ hours per week per SDR, allowing them to focus on more strategic sales activities and boosting response rates by 30%.  

Our seamless CRM integration works with Salesforce, HubSpot, and other systems, streamlining your lead flow.  Furthermore, our automated personalized outreach helps you instantly connect with prospects on LinkedIn, ensuring a more 

Project Structure

In [ ]:
ai_email_gen/
├── app.py
├── lead_product_sample.json
├── requirements.txt
├── Dockerfile

requirements.txt


In [ ]:
langchain
crewai
transformers
huggingface_hub
pandas

Dockerfile

In [ ]:
FROM python:3.10-slim

# 1. Install basic OS dependencies
RUN apt-get update && apt-get install -y git && rm -rf /var/lib/apt/lists/*

# 2. Set working directory
WORKDIR /app

# 3. Copy code and requirements
COPY requirements.txt ./
COPY app.py ./
COPY lead_product_sample.json ./

# 4. Install Python dependencies
RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r requirements.txt

# 5. Set up entry point
CMD ["python", "app.py"]

app.py

In [ ]:
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms.base import LLM
from crewai import Agent, Task, Crew, Process
import pandas as pd

# 1. Data Handling
with open("lead_product_sample.json", "r") as f:
    data = json.load(f)
lead = data['lead']
product = data['product']

# 2. LLM Interface
model_name = "google/gemma-1.1-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
gen_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

class Gemma1BItLLM(LLM):
    def _call(self, prompt, stop=None):
        output = gen_pipe(prompt)[0]['generated_text']
        return output[len(prompt):].strip()
    @property
    def _llm_type(self) -> str:
        return "custom-gemma-1b-it"
llm = Gemma1BItLLM()

# 3. Prompt builder
def build_email_prompt(lead, product):
    prompt = f"""You are an expert SDR assistant.

Given the following lead and product information, generate a highly personalized cold email:

Lead:
- Name: {lead['name']}
- Job Title: {lead['job_title']}
- Company: {lead['company']}
- Interests: {', '.join(lead['interests'])}

Product:
- Name: {product['name']}
- Features: {', '.join(product['features'])}
- Benefits: {', '.join(product['benefits'])}

Requirements:
1. Create an engaging subject line directly addressing the lead's pain points or goals.
2. Write a brief, conversational body making the case for why your product could help {lead['company']}.
3. Reference both lead's interests and product benefits.
4. Close with a call-to-action for a short intro call.

Format:
Subject: <Your Subject Line>
Body:
<Your Email Body>

### Output:
"""
    return prompt

# 4. CrewAI agent
reasoning_agent = Agent(
    role="Email Personalization Reasoner",
    goal="Analyze lead and product data to craft optimized sales email",
    backstory="Handles rapid, high-quality outreach for a SaaS sales team.",
    verbose=True,
    allow_delegation=False
)

email_task = Task(
    description="Generate personalized sales email with engaging subject and body.",
    agent=reasoning_agent,
    expected_output="A two-part result: (1) subject line, (2) personalized email body.",
    output_format="Markdown"
)

crew = Crew(
    agents=[reasoning_agent],
    tasks=[email_task],
    process=Process.sequential,
)

# 5. Run email generation
def llm_executor(prompt):
    return llm(prompt)

email_prompt = build_email_prompt(lead, product)
email_output = llm_executor(email_prompt)

match = re.search(r"Subject:\s*(.*?)\nBody:\s*([\s\S]*)", email_output)
if match:
    subject = match.group(1).strip()
    body = match.group(2).strip()
else:
    subject = "N/A"
    body = email_output.strip()
print(f"\n---\nSubject: {subject}\n\nBody:\n{body}\n---")

# Optionally, save output as CSV for downstream use
df = pd.DataFrame([{"Name": lead["name"], "Subject": subject, "Body": body}])
df.to_csv("generated_email.csv", index=False)

lead_product_sample.json

In [ ]:
{
  "lead": {
    "name": "Jane Doe",
    "job_title": "Head of Sales",
    "company": "Acme Corp",
    "interests": ["B2B SaaS", "lead generation", "CRM automation"]
  },
  "product": {
    "name": "SalesAI Automation Suite",
    "features": [
      "AI-driven lead enrichment",
      "Seamless CRM integration",
      "Automated personalized outreach"
    ],
    "benefits": [
      "Save 10+ hours/week per SDR",
      "Boost response rates by 30%",
      "Instant LinkedIn-to-inbox flows"
    ]
  }
}

Build and Run


In [ ]:
docker build -t ai-email-gen .
docker run --rm ai-email-gen

Optional: Mount Output Folder

In [ ]:
docker run --rm -v $PWD/output:/app/output ai-email-gen